In [ ]:
import re
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup as bs
from vac_name_cleener import vac_name_cleener
from HH_vac_block_parcer import get_info

In [ ]:
vacancy_name = 'sql'

In [ ]:
vacancy_name = vac_name_cleener(vacancy_name)

vacancy_list = []
salary_list = []
min_salary = []
max_salary = []
url_list = []
currency_list = []
source_name_list = []
employer_list = []


In [ ]:
pages = 1  # !!!!!!!!!!!!!!!!!!!!!!!!!


In [ ]:
domain = 'hh.ru'
main_link = 'https://hh.ru/search/vacancy?text='

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) \
     AppleWebKit/537.36 (KHTML, like Gecko) \
     Chrome/76.0.3809.100 Safari/537.36 OPR/63.0.3368.53'
}

In [ ]:
pg_num = '2'

In [ ]:
page = str(pg_num);
page

In [ ]:
link = main_link + vacancy_name + '&page=' + page
link

In [ ]:
html = requests.get(link, headers=headers)
html

In [ ]:
soup = bs(html.text, 'lxml')

In [ ]:
htmlout = str(soup)
with open("soup.html", "w") as file:
    file.write(htmlout)

In [ ]:
vacancy_block = soup.find_all('div', class_='vacancy-serp-item') # 50 vacs was found

In [ ]:
len(vacancy_block)

### * * * * * * 

In [ ]:
vacancy = vacancy_block[8]

In [ ]:
len(vacancy)

In [ ]:
# должность
vacancy.find_all('a', href=True)[0].text

In [ ]:
for i in range(len(vacancy.find_all('a', href=True))):
    print(i, vacancy.find_all('a', href=True)[i].text)

In [ ]:
# работодатель
employers_tags = {"class":"bloko-link bloko-link_secondary", "data-qa":"vacancy-serp__vacancy-employer"}
vacancy.find("a", employers_tags).text

In [ ]:
<span class="vacancy-serp-item__meta-info" data-qa="vacancy-serp__vacancy-address">Саратов</span>

In [ ]:
salary_tags = {'class':'bloko-section-header-3 bloko-section-header-3_lite',
               'data-qa':'vacancy-serp__vacancy-compensation'}
vacancy.find('span', salary_tags) is None

In [ ]:
vacancy.find(class_="bloko-link bloko-link_secondary").text

In [ ]:
vacancy.find('span', salary_tags).text

In [ ]:
salaryblock = vacancy.find('span', salary_tags).text


if salaryblock[:2] == 'от':
    salary_min = ''.join(re.findall(r'[0-9]', salaryblock))
    salary_max = None
elif salaryblock[:2] == 'до':
    salary_min = None
    salary_max = ''.join(re.findall(r'[0-9]', salaryblock))
else:
    salary = salaryblock.split('-')
    if len(salary) == 1:
        salary_min = salary[0]
        salary_max = salary[0]
    else:
        salary_min = ''.join(re.findall(r'[0-9]', salary[0]))
        salary_max = ''.join(re.findall(r'[0-9]', salary[-1]))
salary_min = int(salary_min)
salary_max = int(salary_max)
currency = salaryblock[-4:]      

In [ ]:
salaryblock

In [ ]:
print(salary_min)
print(salary_max)
print(currency)

In [ ]:
type(salary_min)

In [ ]:
link = vacancy.find_all('a', href=True)[0].get('href')

In [ ]:
link

In [ ]:
vacancy.find(class_="bloko-link bloko-link_secondary").text

In [ ]:
vacancy.find({'class':'bloko-link bloko-link_secondary'}).text

# superjob

In [2]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import time

from getpage import getpage
from name_cleener import sj_name_cleener

### Временное
vacancy_name = "c#"
pg_num = 0
###

# создаем запрос и получаем ответ
domain = 'https://www.superjob.ru'
main_link = 'https://www.superjob.ru/vacancy/search/?keywords='
page = str(pg_num)
link = main_link + sj_name_cleener(vacancy_name) + '&page=' + str(page)
print(f'Conneting to page № {pg_num}. {link}')
bsObj = getpage(link)

# создаем список элементов с названиями вакансий и  URL
vacancy_list=[]
url_list=[]

tag = 'div'
class_attrs = {'class':'_3mfro PlM3e _2JVkc _3LJqf'}
vacancy_block = bsObj.findAll(tag, class_attrs)

for vacancy in vacancy_block:
    vacancy_list.append(vacancy.getText())
    url_list.append(domain + vacancy.find('a', href=True)['href'])

# создаем список работодателей
employer_list=[]
employer_block = bsObj.findAll(class_=employer_re)

for i,employer in enumerate(employer_block):
    employer_list.append(employer_block[i].getText())

# создаем список ссылок на источник данных
source_name_list = [domain] * 20

# создаем список элементов с информациям по зарплатам
salary_list = []
tag = 'span'
class_attrs = {'class':'_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}
salary_block = bsObj.findAll(tag, class_attrs)

for i,salary in enumerate(salary_block):
    salary_list.append(salary.getText())
    salary_list[i] = re.sub("\xa0","",salary_list[i])

min_salary = []
max_salary = []
currency_list = []

for salary in salary_list:
    if salary == 'По договорённости':
        salary_min = None
        salary_max = None
        currency = None

    else:
        if salary[:2] == 'от':
            salary_min = int(''.join(re.findall(r'[0-9]', salary)))
            salary_max = None
            currency = re.findall('[\D]+$', salary)[-1]

        elif salary[:2] == 'до':
            salary_min = None
            salary_max = int(''.join(re.findall(r'[0-9]', salary)))
            currency = re.findall('[\D]+$', salary)[-1]

        else:
            currency = re.findall('[\D]+$', salary)[-1]
            salary = re.split(r'—',salary)

            if len(salary) == 1:
                salary_min = int(''.join(re.findall(r'[0-9]', salary[0])))
                salary_max = int(''.join(re.findall(r'[0-9]', salary[0])))

            else:
                salary_min = int(''.join(re.findall(r'[0-9]', salary[0])))
                salary_max = int(''.join(re.findall(r'[0-9]', salary[-1])))
      
    min_salary.append(salary_min)
    max_salary.append(salary_max)
    currency_list.append(currency)

# Создаем результирующий датафрейм
df = pd.DataFrame()
df['vacancy'] = pd.Series(vacancy_list)
df['min_salary'] = pd.Series(min_salary)
df['max_salary'] = pd.Series(max_salary)
df['currency'] = pd.Series(currency_list)
df['url'] = pd.Series(url_list)
df['source'] = pd.Series(source_name_list)
df['employer'] = pd.Series(employer_list)